In [ ]:
import unittest
from unittest.mock import Mock, patch, AsyncMock, MagicMock
import datetime as dt
from freezegun import freeze_time

# Mock all external modules before any imports
mock_ibm_db = MagicMock()
mock_spark = MagicMock()
mock_pyjudo = MagicMock()
mock_earnings = MagicMock()

# Create patch dictionary for all external modules
patches = {
    'ibm_db': mock_ibm_db,
    'pyspark': mock_spark,
    'pyjudo': mock_pyjudo,
    'workspace.ai.libs.data.earnings': mock_earnings,
}

# Apply all patches
for mod, mock in patches.items():
    patch(mod, mock).start()

# Now import your modules
from workspace.ai.libs.ingestion.vespa.ingest import (
    backfill_ingestion_earnings,
    daily_ingestion_earnings,
    get_s3_credentials,
    pull_ingest_update
)
from workspace.ai.libs.ingestion.vespa.vespa_manager import VespaManager

class TestIngest(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        """Set up test fixtures for the entire test class"""
        # Mock Spark context
        cls.mock_spark_context = MagicMock()
        cls.mock_spark_context.get_spark_context.return_value = MagicMock()
        
        # Mock DB2 connection
        cls.mock_db2_conn = MagicMock()
        cls.mock_db2_conn.execute.return_value = MagicMock()
        
        # Apply class-level patches
        cls.patches = [
            patch('workspace.ai.libs.data.spark.get_spark_context', 
                  return_value=cls.mock_spark_context),
            patch('pyjudo.database.db2.connect', 
                  return_value=cls.mock_db2_conn),
            patch('workspace.ai.libs.data.earnings.load_earnings_call_data_vespa',
                  return_value=MagicMock())
        ]
        
        for p in cls.patches:
            p.start()

    def setUp(self):
        """Set up test fixtures"""
        self.schema_id = "test_schema"
        self.s3_bucket_id = "test_bucket"
        self.mock_df = MagicMock()
        self.mock_df.toPandas.return_value = MagicMock()

    @classmethod
    def tearDownClass(cls):
        """Clean up after all tests"""
        for p in cls.patches:
            p.stop()

    @patch('asyncio.run')
    @patch('workspace.ai.libs.ingestion.vespa.ingest.my_dict')
    def test_backfill_ingestion_earnings(self, mock_my_dict, mock_asyncio_run):
        mock_my_dict.items.return_value = [
            ("test_dir", ["2025-01-01", "2025-01-02"])
        ]
        
        backfill_ingestion_earnings(self.schema_id, self.s3_bucket_id)
        
        mock_asyncio_run.assert_called()
        self.assertEqual(mock_asyncio_run.call_count, 1)

    @freeze_time("2025-01-31")
    @patch('asyncio.run')
    @patch('workspace.ai.libs.ingestion.vespa.ingest.job_date_if_none')
    def test_daily_ingestion_earnings(self, mock_job_date, mock_asyncio_run):
        mock_job_date.return_value = "20250131"
        
        daily_ingestion_earnings(self.schema_id, self.s3_bucket_id)
        
        mock_asyncio_run.assert_called_once()
        self.assertIn("2025_ECT/20250131", str(mock_asyncio_run.call_args))

class TestVespaManager(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        """Set up any needed patches"""
        cls.patches = [
            patch('workspace.ai.libs.ingestion.vespa.vespa_manager.S3'),
            patch('workspace.ai.libs.ingestion.vespa.vespa_manager.create_headers'),
            patch('workspace.ai.libs.ingestion.vespa.vespa_manager.create_url')
        ]
        for p in cls.patches:
            p.start()

    def setUp(self):
        self.vespa_manager = VespaManager(
            schema_id="test_schema",
            env="test_env",
            s3_bucket_id="test_bucket",
            creds={"access_key": "test_key", "secret_key": "test_secret"},
            logging_dir="test_logs"
        )
        self.sample_payload = {
            "nativeId": "test123",
            "fields": {
                "tickers_s": ["AAPL", "GOOGL"],
                "year_s": "2025",
                "quarter_s": "Q1",
                "event_time_s": "2025-01-15T10:00:00Z"
            }
        }

    @classmethod
    def tearDownClass(cls):
        for p in cls.patches:
            p.stop()

    @patch('aiohttp.ClientSession')
    async def test_ingest_in_vespa(self, mock_session):
        with patch.object(self.vespa_manager, 'get_ingestion_tracker') as mock_tracker, \
             patch.object(self.vespa_manager, 'get_ingested_native_id_list') as mock_id_list, \
             patch.object(self.vespa_manager, 'get_failed_payloads') as mock_failed:
            
            mock_tracker.return_value = {}
            mock_id_list.return_value = []
            mock_failed.return_value = {}
            
            mock_response = AsyncMock()
            mock_response.json.return_value = {
                "successDocs": [{"documentId": "doc123"}]
            }
            mock_session.return_value.__aenter__.return_value.post.return_value = mock_response
            
            await self.vespa_manager.ingest_in_vespa([self.sample_payload])
            
            mock_session.return_value.__aenter__.return_value.post.assert_called_once()

    def test_update_failed_payloads(self):
        with patch.object(self.vespa_manager, 'get_failed_payloads') as mock_failed, \
             patch.object(self.vespa_manager.s3, 'write_file') as mock_write:
            
            mock_failed.return_value = {}
            
            self.vespa_manager.update_failed_payloads(
                "test123",
                self.sample_payload,
                "test error"
            )
            
            mock_write.assert_called_once()

if __name__ == '__main__':
    unittest.main()